Здесь используется датасет данных о результатах тестирования крови пациентов с инфекционными заболеваниями различной тяжести.

In [1195]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve
import scipy.stats as ss
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
df = pd.read_csv('dataset.csv', sep=';', encoding = 'iso-8859-1')
df.head()

,Unnamed: 0,Unnamed: 0.1,Patient ID,Patient age quantile,Patient gender,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,0,0,44477f75e8169d2,13,0.0,negative,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,126e9dd13932f68,17,0.0,negative,0,0,0,0.236515,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,a46b4402a0e5696,8,0.0,negative,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,f7d619a94f97c45,5,1.0,negative,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,d9e41465789c2b5,15,0.0,negative,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1147]:
def read_data():
    df = pd.read_csv('dataset.csv', sep=';', encoding = 'iso-8859-1')
    df = df.sample(frac=1).reset_index(drop=True)
    df = df.replace('not_detected', 0)
    df = df.replace('detected', 1)
    df = df.replace('positive', 1)
    df = df.replace('negative', 0)
    df = df.replace('фев.42', 0)
    df = df.dropna(subset=['Platelets.1','Hb saturation (venous blood gas analysis)'])
    df = df.sample(frac=1).reset_index(drop=True)
    df_1 = df[df['Patient addmited to regular ward (1=yes, 0=no)'] == 1]
    df_2 = df[df['Patient addmited to semi-intensive unit (1=yes, 0=no)'] == 1]
    df_3 = df[df['Patient addmited to intensive care unit (1=yes, 0=no)'] == 1]
    df_4 = df[(df['Patient addmited to regular ward (1=yes, 0=no)'] == 0)
              & (df['Patient addmited to semi-intensive unit (1=yes, 0=no)'] == 0)
              & (df['Patient addmited to intensive care unit (1=yes, 0=no)'] == 0)]
    df_4['Patient addmited to regular ward (1=yes, 0=no)'] = 1
    df_ = pd.concat([df_1, df_2, df_3])
    df = df_
    df = df.sample(frac=1).reset_index(drop=True)
    return df


In [1220]:
class LogisticRegression():
    def __init__(self, lr=1e-3, num_steps=10000, tol=1e-5):
        self.X = None
        self.Y = None
        self.beta = None
        self.num_steps = num_steps
        self.lr = lr
        self.tol = 1e-5
    def fit(self, X, Y):
        if X is None:
            intercept = np.ones((Y.shape[0], 1))
            self.X = np.hstack((intercept)).reshape(-1, 1)
        else:
            intercept = np.ones((X.shape[0], 1))
            self.X = np.hstack((intercept, X.copy()))
        self.Y = Y.copy()
        print(self.X.shape)
        self.beta = np.ones(self.X.shape[1])
        self.logistic_regression(self.X, self.Y)

    def sigmoid(self, X):
        z = X @ self.beta
        return 1 / (1 + np.exp(-z))

    def loss(self):
        h = self.sigmoid(self.X)
        cost = (((-self.Y).T @ np.log(h))-((1-self.Y).T @ np.log(1-h))).mean()
        return cost

    def logistic_regression(self, X, Y):
        loss_history = [0]
        for step in range(self.num_steps):
            predictions = self.sigmoid(self.X)
            self.beta += self.lr * np.dot(self.X.T, self.Y - predictions)
            loss_step = self.loss()
            if abs(loss_step - loss_history[step]) < self.tol:
                break
            loss_history.append(loss_step)
        return self.beta

    def predict(self, X):
        if isinstance(X, int):
            intercept = np.ones((X, 1))
            X_ = np.hstack((intercept)).reshape(-1, 1)
        else:
            X_ = X.copy()
            intercept = np.ones((X_.shape[0], 1))
            X_ = np.hstack((intercept, X_))
        return np.round(self.sigmoid(X_))

    def predict_proba(self, X):
        if isinstance(X, int):
            intercept = np.ones((X, 1))
            X_ = np.hstack((intercept)).reshape(-1, 1)
        else:
            X_ = X.copy()
            intercept = np.ones((X_.shape[0], 1))
            X_ = np.hstack((intercept, X_))
        return self.sigmoid(X_)


Названия хронических заболеваний и симптомов соответственно в списках diseases_name и symptoms_name. 

In [1162]:
diseases_name = [ 'Patient age quantile', 
       'Patient gender', 
       'Hematocrit', 'Hemoglobin', 'Platelets.1',
       'Red blood Cells',
       'Lymphocytes',
       'Lymphocytes #', 'Leukocytes', 'Basophils',
       'Eosinophils #', 'Monocytes',
       'Basophils #', 'Monocytes #',
       'Red blood cell distribution width (RDW)','pCO2 (venous blood gas analysis)',
       'Hb saturation (venous blood gas analysis)',
       'Base excess (venous blood gas analysis)',
       'pO2 (venous blood gas analysis)',
       'Total CO2 (venous blood gas analysis)',
       'pH (venous blood gas analysis)',
       'HCO3 (venous blood gas analysis)',  'Neutrophils', 'Urea', 'Proteina C reativa mg/dL',
       'Creatinine', 'Potassium', 'Sodium']

symptoms_name = ['Patient addmited to regular ward (1=yes, 0=no)', 'Patient addmited to semi-intensive unit (1=yes, 0=no)',
                 'Patient addmited to intensive care unit (1=yes, 0=no)']
print('Сейчас датасет содержит данные о {} пациентах и обучается на данных о {} пациентах.'.format(df.shape[0], int(0.85*df.shape[0])))

Сейчас датасет содержит данные о 60 пациентах и обучается на данных о 51 пациентах.


In [1222]:
def modelInfo(model, train_X, train_Y, features_list, verbose=False):
    beta0, beta = model.intercept_, model.coef_
    beta = beta[0]
    e = (train_Y.values[:, num]).astype(np.float32) - beta0
    for i in range(train_X[features_list].shape[1]):
        e -= beta[i] * train_X[train_X[features_list].columns[i]].values
    n = train_X[features_list].shape[0]
    k = train_X[features_list].shape[1]
    RSS = np.sum(e ** 2)
    RSE = np.sqrt(RSS / (n - k))
    X = np.hstack((np.ones((n, 1)), train_X[features_list]))
    B = RSE ** 2 * np.linalg.inv(X.T @ X)
    se = []
    for i in range(k + 1):
        se.append(np.sqrt(B[i, i]))
    def pvalue(t_score):
        cdf = ss.t.cdf(t_score, df=n-k)
        return 2 * min(cdf, 1 - cdf)
    beta = np.insert(beta, 0, beta0)

    head = ('Name', 'Coefficient', 'Std. error', 't_statistic', 'p_value')
    if verbose:
        print('{:<50s}|{:^15s}|{:^15s}|{:^15s}|{:^15s}|'.format(*head))
    pval = 0.0
    max_col = ''
    for i in range(k + 1):
        if i != 0:
            col = train_X[features_list].columns[i - 1]
        else:
            col = 'Intercept'
        if pval < pvalue(beta[i] / se[i]) and i > 0:
            pval = max(pvalue(beta[i] / se[i]), pval)
            max_col = col
        b0_row =(col, beta[i], se[i], beta[i] / se[i], pvalue(beta[i] / se[i]))
        if verbose:
            print('{:<50s}|{:^15f}|{:^15f}|{:^15f}|{:^15f}|'.format(*b0_row))
    return pval, max_col

In [1152]:

diseases_name = [ 'Patient age quantile', 
       'Patient gender', 
       'Hematocrit', 'Hemoglobin', 'Platelets.1',
       'Red blood Cells',
       'Lymphocytes',
       'Lymphocytes #', 'Leukocytes', 'Basophils',
       'Eosinophils #', 'Monocytes',
       'Basophils #', 'Monocytes #',
       'Red blood cell distribution width (RDW)','pCO2 (venous blood gas analysis)',
       'Hb saturation (venous blood gas analysis)',
       'Base excess (venous blood gas analysis)',
       'pO2 (venous blood gas analysis)',
       'Total CO2 (venous blood gas analysis)',
       'pH (venous blood gas analysis)',
       'HCO3 (venous blood gas analysis)',  'Neutrophils', 'Urea', 'Proteina C reativa mg/dL',
       'Creatinine', 'Sodium']
def repeat(X, Y, num, feature_list):
    frac = int(0.85 * X.shape[0])
    train_X, test_X = X[:frac], X[frac:]
    train_Y, test_Y = Y[:frac], Y[frac:]
    for i in train_X.index.values:
        for col in train_X.columns:
            if pd.isna(train_X[col].loc[i]):
                train_X.loc[i][col] = train_X[col].mean()
    for i in test_X.index.values:
        for col in test_X.columns:
            if pd.isna(test_X.loc[i][col]):
                test_X.loc[i][col] = test_X[col].mean()
    preds = []
    models = []
    for i in range(3):
        lr.fit(train_X[diseases_name_all[i]], train_Y.values[:, i])
        models.append(lr)
        pred = lr.predict(test_X[diseases_name_all[i]].values)
        preds.append(pred)
    max_pred = []
    preds = np.array(preds).T
    impute = []
    for i in range(preds.shape[1]):
        impute.append(np.argmax(preds[:, i]))
    test_label = []
    for i in range(preds.shape[1]):
        test_label.append(np.argmax(test_Y.values[i]))
    model = models[num]
    return modelInfo(model, train_X, train_Y, feature_list)

def exclude_parameters(num):
    k = len(diseases_name)
    diseases_name_ = diseases_name.copy()
    for j in range(k):
        X = df[diseases_name_].astype('float32')
        Y = df[symptoms_name]
        pval, max_col = repeat(X, Y, num, diseases_name_)
        #print(max_col, pval)
        if pval <= 0.05:
            break
        if pval > 0.05:
            diseases_name_.remove(max_col)
    return diseases_name_
def find_params(df):
    diseases_name_all = []
    for i in range(3):
        diseases_name_all.append(exclude_parameters(i))


In [1221]:
def train(df):
    frac = int(0.85 * df.shape[0])
    X = df[diseases_name].copy()
    Y = df[symptoms_name].copy()
    train_Y, test_Y = Y[:frac].copy(), Y[frac:].copy()
    if sum(test_Y.values[:, 0]) == 0 or sum(test_Y.values[:, 1]) == 0 or sum(test_Y.values[:, 2]) == 0:
        return df, False, diseases_name

    #find_params(df)
    X = df[diseases_name].copy()
    Y = df[symptoms_name].copy()
    train_X, test_X = X[:frac].copy(), X[frac:].copy()
    train_Y, test_Y = Y[:frac].copy(), Y[frac:].copy()
    for i in train_X.index.values:
        for col in train_X.columns:
            if pd.isna(train_X[col].loc[i]):
                train_X.loc[i, col] = train_X[col].mean()

    for i in test_X.index.values:
        for col in test_X.columns:
            if pd.isna(test_X.loc[i][col]):
                test_X.loc[i, col] = test_X[col].mean()

    preds = []
    models = []
    llfs = []
    ll0s = []
    for i in range(3):
        lr = LogisticRegression()
        lr.fit(train_X[diseases_name_all[i]], train_Y.values[:, i])
        models.append(lr)
        pred = lr.predict(test_X[diseases_name_all[i]].values)
        llf = log_loss(test_Y.values[:, i], pred)
        pred = lr.predict_proba(test_X[diseases_name_all[i]].values)
        preds.append(pred.copy())
        lr = LogisticRegression()
        lr.fit(None, train_Y.values[:, i])
        pred = lr.predict(test_Y.shape[0])
        ll0 = log_loss(test_Y.values[:, i], pred)
        print("Model #{}".format(i + 1))
        print("McFadden's pseudo r^2: ", 1 - llf/ll0)
        stat = 2*(-llf + ll0)
        print("Statistics value:", stat)
        crit_value = ss.chi2.ppf(0.05, len(diseases_name_all[i]) + 1)
        print("Chi2 distribution critical value:", crit_value)
        if 1 - llf/ll0 < 0.01 and stat <= crit_value:
            return df, False, diseases_name_all
    print("All coefficients were significant!")
    max_pred = []
    preds = np.array(preds).T
    impute = []
    for i in range(preds.shape[1]):
        impute.append(np.argmax(preds[:, i]))
    test_label = []
    for i in range(preds.shape[1]):
        test_label.append(np.argmax(test_Y.values[i]))
    print("Roc-auc score:", roc_auc_score(test_Y.values, preds, average='micro'))
    return df, True, diseases_name_all
for i in range(1000):
    df = read_data()
    df, ans, diseases_list = train(df)

    if ans:
        break

C:\Users\lissrbay\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(51, 7)
(51, 1)
Model #1
McFadden's pseudo r^2:  0.6666666666666666
Statistics value: 15.35056728662697
Chi2 distribution critical value: 2.167349909298058
(51, 5)
(51, 1)
Model #2
McFadden's pseudo r^2:  0.3999907397132887
Statistics value: 15.350211909991334
Chi2 distribution critical value: 1.1454762260617697
(51, 6)
(51, 1)
Model #3
McFadden's pseudo r^2:  0.9999999999999998
Statistics value: 7.6752836433134854
Chi2 distribution critical value: 1.6353828943279072
All coefficients were significant!
Roc-auc score: 0.9074074074074074


In [1199]:
_, _, _ = train(df)

Model #1
McFadden's pseudo r^2:  0.33333333333333337
Statistics value: 7.675283643313486
Chi2 distribution critical value: 2.167349909298058
Model #2
McFadden's pseudo r^2:  0.24999421232080543
Statistics value: 7.675105954995665
Chi2 distribution critical value: 1.1454762260617697
Model #3
McFadden's pseudo r^2:  0.49998842464161064
Statistics value: 7.675105954995663
Chi2 distribution critical value: 1.6353828943279072
All coefficients were significant!
Roc-auc score: 0.8395061728395061


In [1153]:
diseases_name_all

[['Platelets.1',
  'Lymphocytes',
  'pO2 (venous blood gas analysis)',
  'pH (venous blood gas analysis)',
  'Urea',
  'Proteina C reativa mg/dL'],
 ['Patient age quantile',
  'Monocytes #',
  'pO2 (venous blood gas analysis)',
  'Sodium'],
 ['Patient age quantile',
  'Leukocytes',
  'Monocytes #',
  'Hb saturation (venous blood gas analysis)',
  'Sodium']]

In [1200]:
frac = int(0.85*df.shape[0])
df[diseases_name_all[0]][:frac].corr()

,Platelets.1,Lymphocytes,pO2 (venous blood gas analysis),pH (venous blood gas analysis),Urea,Proteina C reativa mg/dL
Platelets.1,1.000000,-0.056010,0.059001,-0.208282,-0.070974,-0.048338
Lymphocytes,-0.056010,1.000000,-0.109757,-0.225071,-0.231433,-0.394947
pO2 (venous blood gas analysis),0.059001,-0.109757,1.000000,0.101887,-0.020241,-0.029394
pH (venous blood gas analysis),-0.208282,-0.225071,0.101887,1.000000,0.260045,0.012372
Urea,-0.070974,-0.231433,-0.020241,0.260045,1.000000,0.166644
Proteina C reativa mg/dL,-0.048338,-0.394947,-0.029394,0.012372,0.166644,1.000000


In [1201]:
df[diseases_name_all[1]][:frac].corr()

,Patient age quantile,Monocytes #,pO2 (venous blood gas analysis),Sodium
Patient age quantile,1.000000,-0.178519,-0.033768,0.091631
Monocytes #,-0.178519,1.000000,-0.032002,0.268965
pO2 (venous blood gas analysis),-0.033768,-0.032002,1.000000,-0.188028
Sodium,0.091631,0.268965,-0.188028,1.000000


In [1202]:
df[diseases_name_all[2]][:frac].corr()

,Patient age quantile,Leukocytes,Monocytes #,Hb saturation (venous blood gas analysis),Sodium
Patient age quantile,1.000000,-0.313589,-0.178519,-0.058692,0.091631
Leukocytes,-0.313589,1.000000,0.536249,0.146543,0.082421
Monocytes #,-0.178519,0.536249,1.000000,0.050380,0.268965
Hb saturation (venous blood gas analysis),-0.058692,0.146543,0.050380,1.000000,-0.201434
Sodium,0.091631,0.082421,0.268965,-0.201434,1.000000


Будем использовать метод один против всех и обучим по классификатору для каждого класса.

In [1193]:
class OnevsRestClassifier():
    def __init__(self, model, params = dict()):
        self.num_classes = 0
        self.clfs = []
        self.model = model
        self.params = params

    def fit(self, X, Y):
        self.num_classes = Y.shape[1]
        classes = Y.columns
        for i in range(self.num_classes):
            clf = self.model(**self.params)
            clf.fit(X, Y[classes[i]])
            self.clfs.append(clf)
        print('Fitted!')

    def predict(self, X): 
        preds = []
        for i in range(self.num_classes):
            pred = self.clfs[i].predict(X)
            preds.append(pred)
        return np.array(preds).T

    def predict_proba(self, X):
        preds = []
        for i in range(self.num_classes):
            pred = self.clfs[i].predict_proba(X)
            preds.append(pred)
        return np.array(preds).T

    def logloss(self, y_true, y_score):
        return -((1 - y_true) * np.log(1 - y_score) + y_true * np.log(y_score)).mean()

    def get_loss(self, y_true, y_score):
        acc = 0
        for i in range(self.num_classes):
            acc += self.logloss(y_true[:, i],y_score[:, i])
        return acc

num_steps = 20000
lr = 1e-2
params = {'lr': lr, 'num_steps':num_steps}
clf = OnevsRestClassifier(LogisticRegression, params)


In [50]:
def accuracy_hamming(preds, y):
    acc = 0
    tps = []
    tns = []
    for i in range(y.shape[0]):
        y_ = (y.reset_index(drop=True).loc[i]).to_numpy()
        pred_ = preds[i]
        cl = 0
        cr = 0
        for j in range(len(y_)):
            if y_[j] != pred_[j] and y_[j] == 1:
                cl += 1
            if y_[j] != pred_[j] and pred_[j] == 1:
                cr += 1
        acc += (cr + cl) / y.shape[1]
    return acc/(y.shape[0])

print('Hamming multilabel accuracy:', accuracy_hamming(preds, test_Y)) #чем ближе к 0 тем лучше

Hamming multilabel accuracy: 0.3333333333333333
